<a href="https://colab.research.google.com/github/aaaksenova/RussianConstructicon_NLP_tools/blob/change/Dublicate_Search_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поиск дубликатов конструкций

С помощью этого кода можно проверять не дублируют ли новые конструкции то, что уже есть в конструктиконе

by Анна Аксенова (aksanna_a@mail.ru)

In [1]:
! pip install conllu
! pip install pymorphy2

     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 8.2 MB 47.7 MB/s 


In [2]:
import re
import pandas as pd
from pprint import pprint
from conllu import parse
from conllu import parse_incr, parse_tree_incr
import sqlite3
import numpy as np
import pymorphy2
import nltk
nltk.download('punkt')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
def get_all_types(construction):
  
    """
    Эта функция получает на вход формулу конструкции 
    и выдает все вариативные формы конструкции
    """

    constructions = []
    constructions.append([construction])
    new = []
    if '(' in construction:
        constructions.append([re.sub(r'\(.+?\)', '', construction, count=1), re.sub(r'\((.+?)\)', r'\1', construction, count=1)])
        if '(' in construction:
            new = []
            for c in constructions[-1]:
                new.extend([re.sub(r'\(.+?\)', '', c, count=1), re.sub(r'\((.+?)\)', r'\1', c, count=1)])
            if new:
                constructions.append(new)
    if '/' in construction:
        new = []
        for c in constructions[-1]:
            flag = []
            optional = re.findall(r'([А-Яа-яЁёA-Za-z\-]+)/([А-Яа-яЁёA-Za-z-]+)/*([А-Яа-яЁёA-Za-z-]+)*', c)
            for word in optional[0]:
                if word:
                   flag.append(re.sub(r'([А-Яа-яЁёA-Za-z\-]+)/([А-Яа-яЁёA-Za-z-]+)/*([А-Яа-яЁёA-Za-z-]+)*', word, c, count=1))
            if len(optional) > 1:
                new_1 = []
                for word in optional[1]:
                    if word:
                        new_1.extend([re.sub(r'([А-Яа-яЁёA-Za-z\-]+)/([А-Яа-яЁёA-Za-z-]+)/*([А-Яа-яЁёA-Za-z-]+)*', word, i, count=1) for i in flag])
                new.extend(new_1)
            else:
                new.extend(flag)
    if not new:
        new = [construction]

    constructions.append(new)
    final_list = list(set([i.replace('XP', '').replace('Cop', '').replace('Crd', '').replace('Ord', '').replace('Cl', '') 
                          for i in constructions[-1]]))
    return final_list

In [5]:
get_all_types('NP-Nom (как) давай/пора/уже VP-Inf/Cl (вот)')

['NP-Nom  давай VP-Inf вот',
 'NP-Nom как пора VP-Inf ',
 'NP-Nom  пора VP-Inf вот',
 'NP-Nom  давай VP-Inf ',
 'NP-Nom  давай  ',
 'NP-Nom  пора  вот',
 'NP-Nom как уже VP-Inf ',
 'NP-Nom как уже VP-Inf вот',
 'NP-Nom как уже  вот',
 'NP-Nom  уже  вот',
 'NP-Nom как давай  вот',
 'NP-Nom  уже VP-Inf ',
 'NP-Nom как давай VP-Inf ',
 'NP-Nom  уже  ',
 'NP-Nom  уже VP-Inf вот',
 'NP-Nom  давай  вот',
 'NP-Nom как пора  вот',
 'NP-Nom как давай VP-Inf вот',
 'NP-Nom как пора  ',
 'NP-Nom как уже  ',
 'NP-Nom как пора VP-Inf вот',
 'NP-Nom как давай  ',
 'NP-Nom  пора  ',
 'NP-Nom  пора VP-Inf ']

## Поиск дубликатов

In [ ]:
df = pd.read_csv('Название файла Эксель', sep=';')  
# Файл должен иметь следующий формат: Столбец Exist с конструкциями, которые есть в конструктиконе 
# и столбец Potential с потенциальными новыми конструкциями
df

,Exist,Potential
0,(а) как же NP-Nom?,абы где 3
1,(а) при чём здесь/тут Cop NP-Nom?,абы как 50
2,"(а) Noun-Nom/кто (же) (PronPers-3.Acc) знает, Cl",абы куда 1
3,(а/так) что насчёт XP?,баш на баш 16
4,NP-Nom Cop похожий-Short на NP-Acc (NP-Ins/по ...,без боя 316
...,...,...
4106,NaN,V не щадя себя
4107,NaN,V сжав зубы
4108,NaN,"V, и ну бегать!"
4109,NaN,"V, не сказав (Х-у) ни слова"


In [ ]:
exist = []
for i in df.Exist.to_list()[:2261]:
    try:
        exist.extend(get_all_types(i))
    except:
        pass
clean = []
for i in exist:
    if isinstance(i, str):
       clean.append(re.sub(r'\s+', ' ', re.sub(r'[A-Za-z\-\(\)/?!.–~,1234567890]', '', i).strip()))
    else:
        for j in i:
            clean.append(re.sub(r'\s+', ' ', re.sub(r'[A-Za-z\-\(\)/?!.–~,1234567890]', '', j).strip()))

In [ ]:
p = df.Potential.to_list()
potential = []
for i in p:
    if isinstance(i, str):
        potential.append(re.sub(r'\s+', ' ', re.sub(r'[A-Za-z\-\(\)/?!.–~,1234567890]', '', i).strip()))

In [ ]:
exist_dub = []
potential_dub = []
non_dublicate = []

for p in potential:
    for e in clean:
        if len(p) > 2:
            if p in e:
                potential_dub.append(p)
                exist_dub.append(e)
            else:
                non_dublicate.append(p)

In [ ]:
df = pd.DataFrame({'existing_construction': exist_dub, 'potential_dublicate': potential_dub})

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df = pd.DataFrame({'non_dublicate': non_dublicate})

,non_dublicate
0,абы где
1,абы где
2,абы где
3,абы где
4,абы где
...,...
12539000,не подобает у
12539001,не подобает у
12539002,не подобает у
12539003,не подобает у


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.to_csv('non_dublicate.csv')

In [ ]:
df.to_csv('potential_dublicates.csv', sep=';')